# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [77]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

# import statements
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import confusion_matrix, fbeta_score, classification_report, make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from scipy.stats import hmean
from scipy.stats.mstats import gmean
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

import pickle

import warnings
warnings.filterwarnings('ignore')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///SQL_fig8.db')


In [3]:

df = pd.read_sql('messages', engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'genre'])


In [4]:
df.shape

(26216, 39)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 39 columns):
id                        26216 non-null int64
message                   26216 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null int64
missing_people            26216 non-null in

In [6]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

### 2. Write a tokenization function to process your text data

In [78]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    # remove stop words
    STOPWORDS = stopwords.words("english")
    tokens = [word for word in tokens if word not in STOPWORDS]
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


In [79]:

for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'haiti', ',', 'rest', 'country', 'today', 'tonight'] 



In [8]:

for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'of', 'haiti', ',', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [80]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=101)

model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [82]:
def overall_evaluation(model, X_test, y_test):
    multi_predictions = pd.DataFrame(model.predict(X_test))
    multi_predictions.columns = y_test.columns.copy()

    eval_list = []
    for column in multi_predictions:
        # set each value to be the last character of the string
        #confusion_mat = confusion_matrix(y_test[column], multi_predictions[column])
        report = classification_report(y_test[column],multi_predictions[column])
        accuracy = accuracy_score(y_test[column],multi_predictions[column])
        precision = precision_score(y_test[column],multi_predictions[column], average='weighted')
        recall = recall_score(y_test[column],multi_predictions[column], average='weighted')
        f1 = f1_score(y_test[column],multi_predictions[column], average='weighted')
        print("Label:", column)
        print(report)
        eval_list.append([precision, recall, accuracy, f1])
        #precision_list.append(precision)
        #recall_list.append(recall)
        #f1_list.append(f1)

        print("-----------------------------------------------------------------------")

    evaluation = pd.DataFrame(eval_list)
    evaluation.columns = ['precision','recall','accuracy','f1_score']
    #evaluation['recall'] = recall_list
    #evaluation['accuracy'] = accuracy_list
    #evaluation['f1_score'] = f1_list
    print(evaluation)
    print("*******Overall Evaluation*******\nPrecision:{:.2f}\tRecall:{:.2f}\nAccuracy:{:.2f}\tF1 Score:{:.2f}".format(
        np.mean(evaluation.precision), np.mean(evaluation.recall), 
        np.mean(evaluation.accuracy), np.mean(evaluation.f1_score)))
    
    return evaluation

In [83]:
first_evaluation = overall_evaluation(model, X_test, y_test)


Label: related
             precision    recall  f1-score   support

          0       0.62      0.39      0.48      1250
          1       0.82      0.92      0.87      3955
          2       0.25      0.03      0.05        39

avg / total       0.77      0.79      0.77      5244

-----------------------------------------------------------------------
Label: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4348
          1       0.78      0.40      0.53       896

avg / total       0.87      0.88      0.86      5244

-----------------------------------------------------------------------
Label: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5217
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5244

-----------------------------------------------------------------------
Label: aid_related
             precision  

In [84]:


print("*******Overall Evaluation*******\nPrecision:{:.2f}\tRecall:{:.2f}\nAccuracy:{:.2f}\tF1 Score:{:.2f}".format(
    np.mean(first_evaluation.precision), np.mean(first_evaluation.recall), 
    np.mean(first_evaluation.accuracy), np.mean(first_evaluation.f1_score)))


*******Overall Evaluation*******
Precision:0.93	Recall:0.94
Accuracy:0.94	F1 Score:0.93


### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(RandomForestClassifier()))])


# hyper-parameter grid
parameters = {'vect__ngram_range': ((1, 1), (1, 2)),
              'vect__max_df': (0.50, 0.75, 1.0),
              'tfidf__use_idf': (True, False)
              }

# create model
cv = GridSearchCV(estimator=pipeline,
        param_grid=parameters,
        verbose=3,
        cv=3)
#return model


In [28]:

model_1 = cv.fit(X_train, y_train)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 1) .
[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.22915176655700187, total=  50.4s
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 1) .


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.221570590759548, total=  49.2s
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 1) .


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.0min remaining:    0.0s


[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.22188841201716739, total=  49.4s
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 2) .
[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.2251466170791017, total= 2.2min
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 2) .
[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.21842368759834072, total= 2.1min
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 2) .
[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__ngram_range=(1, 2), score=0.22989985693848355, total= 1.9min
[CV] tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.2207123444428551, total=  50.0s
[CV] tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.217422

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 58.3min finished


In [29]:
model_1.best_params_

{'tfidf__use_idf': True, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 2)}

In [30]:
model_1.best_score_

0.22911501049017738

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [42]:

second_evaluation = overall_evaluation(model_1, X_test, y_test)


Label: related
             precision    recall  f1-score   support

          0       0.61      0.38      0.46      1250
          1       0.82      0.92      0.87      3955
          2       0.00      0.00      0.00        39

avg / total       0.76      0.79      0.77      5244

-----------------------------------------------------------------------
Label: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4348
          1       0.82      0.42      0.55       896

avg / total       0.88      0.88      0.87      5244

-----------------------------------------------------------------------
Label: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5217
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5244

-----------------------------------------------------------------------
Label: aid_related
             precision  

In [49]:

print("*******Overall Evaluation*******\nPrecision:{:.2f}\tRecall:{:.2f}\nAccuracy:{:.2f}\tF1 Score:{:.2f}".format(
    np.mean(first_evaluation.precision), np.mean(first_evaluation.recall), 
    np.mean(first_evaluation.accuracy), np.mean(first_evaluation.f1_score)))


*******Overall Evaluation*******
Precision:0.93	Recall:0.94
Accuracy:0.94	F1 Score:0.93


### f1 score and accuracy has not improved compare to previous run using gridsearch cv

In [20]:


print("*******Overall Evaluation*******\nPrecision:{:.2f}\tRecall:{:.2f}\nAccuracy:{:.2f}\tF1 Score:{:.2f}".format(
    np.mean(second_evaluation.precision), np.mean(second_evaluation.recall), 
    np.mean(second_evaluation.accuracy), np.mean(second_evaluation.f1_score)))
                                                                                     


*******Overall Evaluation*******
Precision:0.93	Recall:0.94
Accuracy:0.94	F1 Score:0.93


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [47]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [48]:
pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())])),
            ('starting_verb', StartingVerbExtractor())])),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])

parameters = {'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
              'features__text_pipeline__tfidf__use_idf': (True, False)}

#scorer1 = make_scorer(np.mean(second_evaluation.f1_score))

cross_validation = GridSearchCV(pipeline, param_grid=parameters, verbose = 3, n_jobs=-1)

model_3 = cross_validation.fit(X_train, y_train)


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1), score=0.28965813188385064, total= 1.0min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1), score=0.27335145186668575, total= 1.0min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  3.4min remaining:    0.0s


[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1), score=0.27739628040057224, total= 1.0min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2), score=0.2866542697754256, total= 1.1min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2), score=0.26720068659705337, total= 1.1min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2), score=0.28025751072961375, total= 1.1min
[CV] features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 21.0min finished


In [58]:
third_evaluation = overall_evaluation(model_3, X_test, y_test)


Label: related
             precision    recall  f1-score   support

          0       0.66      0.54      0.60      1250
          1       0.86      0.92      0.89      3955
          2       0.85      0.28      0.42        39

avg / total       0.81      0.82      0.82      5244

-----------------------------------------------------------------------
Label: request
             precision    recall  f1-score   support

          0       0.92      0.97      0.95      4348
          1       0.82      0.61      0.70       896

avg / total       0.91      0.91      0.90      5244

-----------------------------------------------------------------------
Label: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5217
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5244

-----------------------------------------------------------------------
Label: aid_related
             precision  

In [51]:
print("*******Overall Evaluation*******\nPrecision:{:.2f}\tRecall:{:.2f}\nAccuracy:{:.2f}\tF1 Score:{:.2f}".format(
    np.mean(first_evaluation.precision), np.mean(first_evaluation.recall), 
    np.mean(first_evaluation.accuracy), np.mean(first_evaluation.f1_score)))


*******Overall Evaluation*******
Precision:0.93	Recall:0.94
Accuracy:0.94	F1 Score:0.93


### f1 score and accuracy has improved by 1 % compare to previous run using gridsearch cv, I will consider this model as Final Model, if next model does not improve any further.

## Let's Try AdaboostClassifier

In [54]:
pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())])),
            ('starting_verb', StartingVerbExtractor())])),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

parameters = {'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
              'features__text_pipeline__tfidf__use_idf': (True, False)}

#scorer1 = make_scorer(np.mean(second_evaluation.f1_score))

cross_validation = GridSearchCV(pipeline, param_grid=parameters, verbose = 3, n_jobs=-1)

model_4 = cross_validation.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1), score=0.24245458446574167, total= 2.8min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1), score=0.22743527392361607, total= 2.7min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  7.0min remaining:    0.0s


[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1), score=0.2330472103004292, total= 2.7min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2), score=0.23201258761264482, total= 6.0min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2), score=0.22228579602345874, total= 6.0min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2), score=0.2301859799713877, total= 6.0min
[CV] features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__t

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 60.3min finished


In [57]:
fourth_evaluation = overall_evaluation(model_4, X_test, y_test)


Label: related
             precision    recall  f1-score   support

          0       0.56      0.25      0.34      1250
          1       0.79      0.94      0.86      3955
          2       0.33      0.08      0.12        39

avg / total       0.73      0.77      0.73      5244

-----------------------------------------------------------------------
Label: request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      4348
          1       0.78      0.56      0.65       896

avg / total       0.89      0.90      0.89      5244

-----------------------------------------------------------------------
Label: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5217
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5244

-----------------------------------------------------------------------
Label: aid_related
             precision  

In [60]:
print("*******Overall Evaluation*******\nPrecision:{:.2f}\tRecall:{:.2f}\nAccuracy:{:.2f}\tF1 Score:{:.2f}".format(
    np.mean(third_evaluation.precision), np.mean(third_evaluation.recall), 
    np.mean(third_evaluation.accuracy), np.mean(third_evaluation.f1_score)))


*******Overall Evaluation*******
Precision:0.94	Recall:0.95
Accuracy:0.95	F1 Score:0.94


#### We can see No additional improvement in overall accuracy and F1 Score.

# Lets compare all categories for outcome of 36 classification
### Below we can see different in each category, If a value is Positive then Fourth Model(Adaboost) is doing well compare Third model(OneVsRestClassifier(LinearSVC())).

In [70]:
final_eval = fourth_evaluation*100 - third_evaluation*100
final_eval['Categories'] = pd.DataFrame(y_test.columns)
final_eval

,precision,recall,accuracy,f1_score,Categories
0,-8.129523,-5.644546,-5.644546,-8.503936,related
1,-1.473329,-1.277651,-1.277651,-1.454719,request
2,-0.000685,-0.133486,-0.133486,-0.066787,offer
3,-1.346423,-1.620900,-1.620900,-2.037108,aid_related
4,0.062810,-0.038139,-0.038139,0.253387,medical_help
5,-0.566863,-0.343249,-0.343249,-0.021963,medical_products
6,0.028294,0.057208,0.057208,0.379680,search_and_rescue
7,-0.628888,-0.247902,-0.247902,-0.078894,security
8,0.291840,0.133486,0.133486,0.265835,military
9,0.000000,0.000000,0.000000,0.000000,child_alone


#### As we can see there are very few categories where values are in positive side, but with very minor difference, mostly below 1%.  

#### Next, we can see "related" category f1 score is -8%, which mean third model is performing better than fourth in this category, and we can see similar results in categories like <u>request, aid_related and weather_related </u> . Therefore, I would choose Third Model as my Final model for this problem.

### 9. Export your model as a pickle file

In [71]:
pickle.dump(model_3, open('ML_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.